In [5]:
from recipe_rec.data_loader import get_recipes
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle

In [9]:
bert_encoder = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# load the full dataset
full_dataset = get_recipes("./data/recipes.csv")

print(f"Loaded {full_dataset.shape} recipes...")

# turn ingredients back into a string
full_dataset["RecipeIngredientParts"] = full_dataset["RecipeIngredientParts"].str.join(
    " "
)

print("Encoding ingredients to SBERT vectors:")

encoded_ingredients = bert_encoder.encode(
    full_dataset["RecipeIngredientParts"].values, show_progress_bar=True
)
with open("sbert_ingredient_vectors.pkl", "wb") as f:

    pickle.dump(encoded_ingredients, f)

Loaded (511626, 28) recipes...
Encoding ingredients to SBERT vectors:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
from glove import Glove, Corpus

/home/jake/diss/.venv/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jake/diss/.venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jake/diss/.venv/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jake/diss/.venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [5]:
from __future__ import print_function
import argparse
import pprint
import gensim

from glove import Glove
from glove import Corpus


def read_corpus(filename):

    delchars = [chr(c) for c in range(256)]
    delchars = [x for x in delchars if not x.isalnum()]
    delchars.remove(" ")
    delchars = "".join(delchars)

    print(delchars)
    # with open(filename, 'r') as datafile:
    #     for line in datafile:
    #         yield line.lower().translate(None, delchars).split(' ')


def read_wikipedia_corpus(filename):

    # We don't want to do a dictionary construction pass.
    corpus = gensim.corpora.WikiCorpus(filename, dictionary={})

    for text in corpus.get_texts():
        yield text

In [6]:
read_corpus("akjhd")

 	
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ ¡¢£¤¥¦§¨©«¬­®¯°±´¶·¸»¿×÷


In [ ]:
# # Set up command line parameters.
# parser = argparse.ArgumentParser(description='Fit a GloVe model.')

# parser.add_argument('--create', '-c', action='store',
#                     default=None,
#                     help=('The filename of the corpus to pre-process. '
#                             'The pre-processed corpus will be saved '
#                             'and will be ready for training.'))
# parser.add_argument('-wiki', '-w', action='store_true',
#                     default=False,
#                     help=('Assume the corpus input file is in the '
#                             'Wikipedia dump format'))
# parser.add_argument('--train', '-t', action='store',
#                     default=0,
#                     help=('Train the GloVe model with this number of epochs.'
#                             'If not supplied, '
#                             'We\'ll attempt to load a trained model'))
# parser.add_argument('--parallelism', '-p', action='store',
#                     default=1,
#                     help=('Number of parallel threads to use for training'))
# parser.add_argument('--query', '-q', action='store',
#                     default='',
#                     help='Get closes words to this word.')

# args = {}

# if args.create:
#     # Build the corpus dictionary and the cooccurrence matrix.
#     print('Pre-processing corpus')

#     if args.wiki:
#         print('Using wikipedia corpus')
#         get_data = read_wikipedia_corpus
#     else:
#         get_data = read_corpus

#     corpus_model = Corpus()
#     corpus_model.fit(get_data(args.create), window=10)
#     corpus_model.save('corpus.model')

#     print('Dict size: %s' % len(corpus_model.dictionary))
#     print('Collocations: %s' % corpus_model.matrix.nnz)

# if args.train:
#     # Train the GloVe model and save it to disk.

#     if not args.create:
#         # Try to load a corpus from disk.
#         print('Reading corpus statistics')
#         corpus_model = Corpus.load('corpus.model')

#         print('Dict size: %s' % len(corpus_model.dictionary))
#         print('Collocations: %s' % corpus_model.matrix.nnz)

#     print('Training the GloVe model')

#     glove = Glove(no_components=100, learning_rate=0.05)
#     glove.fit(corpus_model.matrix, epochs=int(args.train),
#                 no_threads=args.parallelism, verbose=True)
#     glove.add_dictionary(corpus_model.dictionary)

#     glove.save('glove.model')

# if args.query:
#     # Finally, query the model for most similar words.
#     if not args.train:
#         print('Loading pre-trained GloVe model')
#         glove = Glove.load('glove.model')

#     print('Querying for %s' % args.query)

#     pprint.pprint(glove.most_similar(args.query, number=10))

In [10]:
from recipe_rec.data_loader import get_recipes

df = get_recipes("./data/recipes.csv")

In [19]:
recipe_ingredients = df["RecipeIngredientParts"].apply(lambda tup: list(tup))

In [37]:
VECTOR_SIZE = 100
NUM_EPOCHS = 25
LEARNING_RATE = 0.001
WINDOW = 5

corpus = Corpus()
corpus.fit(recipe_ingredients, window=WINDOW)
corpus.save("corpus.model")


glove = Glove(no_components=VECTOR_SIZE, learning_rate=LEARNING_RATE)
glove.fit(corpus.matrix, epochs=NUM_EPOCHS, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save("glove.model")

Performing 25 training epochs with 2 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24


In [76]:
glove.most_similar("granulated sugar")

[('cinnamon', 0.9975804099733946),
 ('all-purpose flour', 0.9971611296325676),
 ('vanilla extract', 0.9971607104886437),
 ('egg', 0.9968816220333483)]